In [19]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nelly.loh/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
## Types of Financial Crimes & Keywords
# fraud
# electronic crime
# money laundering
# terrorist financing
# bribery
# corruption
# market abuse 
# information security
# insider dealing
# identity theft
# money mule
# national crime agency
# tipping off

# References [Keywords]
# https://www.int-comp.org/careers/your-career-in-financial-crime-prevention/what-is-financial-crime/
# https://www.shireleasing.co.uk/wp-content/uploads/2020/01/Financial-Crime-Key-Words.pdf

# References [Methodology]
# https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c

In [8]:
news_search = ['fraud', 'electronic+crime', 'money+laundering', 'terrorist+financing', 'bribery', 'corruption', 'market+abuse', 'information+security', 'insider+dealing', 'identity+theft', 'money+mule', 'national+crime+agency', 'tipping+off']

In [13]:
def generate_link(identifier_dict, no_of_articles):
    link_start = "https://www.google.com/search?q="
    link_end = "&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2" 
    link_query = ""
    for identifier in identifier_dict.values():        
        link_query += identifier.replace(' ','+') + '+'
    return link_start + link_query + link_end + "&num=" + str(no_of_articles)

def article_extraction(link):
    article = Article(link)
    article.download()
    try:
        article.parse()
    except:
        pass
    return article.text

def sentiment_analysis(text):
    vader = SentimentIntensityAnalyzer()
    return vader.polarity_scores(text)

In [14]:
# Main Function
def search_articles_on_text(search_text, no_of_articles):
    def generate_link(search_text, no_of_articles):
        link_start = "https://www.google.com/search?q=" + search_text
        link_end = "&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2" 
        link_query = ""
        return link_start + link_query + link_end + "&num=" + str(no_of_articles)
    
    link = generate_link(search_text, no_of_articles)
    req = Request(link, headers = {'User-Agent': 'Mozilla/5.0'})

    webpage = urlopen(req).read()

    links = []
    with requests.Session() as c:
        soup = BeautifulSoup(webpage, 'html5lib')
        #print(soup)
        for item in soup.find_all('div', attrs = {'class': "ZINbbc xpd O9g5cc uUPGi"}):
            current_dict = {}
            raw_link = (item.find('a', href = True)['href'])
            link = (raw_link.split("/url?q=")[1]).split('&sa=U&')[0]
            title = (item.find('div',attrs = {'class': 'BNeawe vvjwJb AP7Wnd'})).text
            description  = (item.find('div',attrs = {'class': 'BNeawe s3v9rd AP7Wnd'}).get_text())
            time = description.split(" · ")[0]
            #print(description)
            descript = description.split(" · ")[1]
            current_dict['title'] = title
            current_dict['time'] = time
            current_dict['description'] = descript
            current_dict['link'] = link
            current_dict['text'] = article_extraction(link)
            current_dict['sentiment'] = sentiment_analysis(article_extraction(link))
            links.append(current_dict)
    return links

In [15]:
df = pd.DataFrame()
for search in news_search:
    articles = search_articles_on_text(search, 20)
    df = df.append(articles)

/Users/nelly.loh/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Users/nelly.loh/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [16]:
df

,title,time,description,link,text,sentiment
0,Forum: How can customers be accountable for OT...,9 hours ago,Bank customers routinely disclose their credit...,https://www.straitstimes.com/opinion/forum/for...,"Reading the article, ""Hackers steal OTPs sent ...","{'neg': 0.171, 'neu': 0.744, 'pos': 0.085, 'co..."
1,Forum: Bank kept asking me to pay for OTP frau...,9 hours ago,When I asked the officer in charge about the p...,https://www.straitstimes.com/opinion/forum/for...,"I was a victim of the fraud reported in ""Hacke...","{'neg': 0.112, 'neu': 0.789, 'pos': 0.098, 'co..."
2,Popular Chinese anti-fraud app slammed by West...,2 days ago,An anti-fraud app in China has recently gone v...,https://www.globaltimes.cn/page/202109/1234296...,Photo: VCG\n\nAn anti-fraud app in China has r...,"{'neg': 0.067, 'neu': 0.889, 'pos': 0.044, 'co..."
3,OCBC Bank's fraud detection recovers SG$8 mill...,2 weeks ago,While cybercriminals still manage to avoid the...,https://techwireasia.com/2021/08/ocbc-banks-fr...,Fraud detection solutions continue to grow in ...,"{'neg': 0.177, 'neu': 0.755, 'pos': 0.068, 'co..."
4,Giles McNeill: Ex-West Lindsey council leader ...,11 hours ago,A former Conservative council leader has been ...,https://www.bbc.com/news/uk-england-lincolnshi...,"The allegations against Giles McNeill, 39, rel...","{'neg': 0.276, 'neu': 0.724, 'pos': 0.0, 'comp..."
...,...,...,...,...,...,...
15,Man fined after being caught on covert CCTV du...,18 hours ago,We will not tolerate fly-tipping and this show...,https://www.harboroughmail.co.uk/news/crime/ma...,A man who was caught on a covert CCTV dumping ...,"{'neg': 0.147, 'neu': 0.778, 'pos': 0.076, 'co..."
16,Former Theranos employee describes being stars...,2 days ago,"Cheung, a lab worker, joined Theranos in 2013 ...",https://www.kake.com/story/44732028/former-the...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound..."
17,Disney's True Prince Is a Little Boy Tipping H...,1 day ago,"Bernardo, who is about 4 years old, starts out...",https://insidethemagic.net/2021/09/disneys-tru...,"Who is Disney’s true prince? Well, the officia...","{'neg': 0.034, 'neu': 0.828, 'pos': 0.138, 'co..."
18,Earth’s tipping points could be closer than we...,1 week ago,When they are pushed too far out of one equili...,https://www.theguardian.com/commentisfree/2021...,If there’s one thing we know about climate bre...,"{'neg': 0.086, 'neu': 0.863, 'pos': 0.051, 'co..."


In [20]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

db = df['text'].tolist()
array_text = []

for sentence in db:
    tokenized_words = word_tokenize(sentence)
    tokenized_sentence = []
    for word in tokenized_words:
        tokenized_sentence.append(lemmatizer.lemmatize(word.lower()))
    tokenized_sentence = " ".join(tokenized_sentence)
    sentence_clean = tokenized_sentence.replace("\n", " ").replace("\'", "")
    array_text.append(sentence_clean)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nelly.loh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
# YAKE

In [22]:
import yake
kw_extractor = yake.KeywordExtractor()
language = "en"
max_ngram_size = 1
deduplication_threshold = 0.7 #repetition of words is allowed in keywords
numOfKeywords = 5
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
for j in range(len(array_text)):
    keywords = custom_kw_extractor.extract_keywords(array_text[j])
    print("Keywords of article", str(j+1), "\n", keywords)

Keywords of article 1 
 [('bank', 0.009580758678945114), ('customer', 0.0125814314288408), ('affected', 0.017038479047972205), ('fake', 0.019902667508421897), ('goodwill', 0.021911478309918516)]
Keywords of article 2 
 [('bank', 0.007255104617832941), ('card', 0.01491671324204907), ('transaction', 0.01491671324204907), ('asked', 0.018474760491786), ('fraud', 0.0200889351946492)]
Keywords of article 3 
 [('app', 0.0022385481623138394), ('china', 0.005668249373609649), ('anti-fraud', 0.008980718356741215), ('financial', 0.008980718356741215), ('time', 0.009739769929664396)]
Keywords of article 4 
 [('fraud', 0.004011618237869643), ('detection', 0.005176527184123416), ('bank', 0.006437861783023376), ('customer', 0.0092755396003302), ('activity', 0.009492853842463424)]
Keywords of article 5 
 [('mcneill', 0.09568045026443411), ('relate', 0.09568045026443411), ('employee', 0.09568045026443411), ('fraud', 0.09568045026443411), ('allegation', 0.15831692877998726)]
Keywords of article 6 
 [('c

In [156]:
## KEYBERT

In [157]:
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
for j in range(len(array_text)):
    keywords = kw_extractor.extract_keywords(array_text[j], stop_words='english')
    print("Keywords of article", str(j+1), "\n", keywords)

Keywords of article 1 
 [('biotech', 0.2538), ('billionaire', 0.2169), ('tuesday', 0.1856), ('cancer', 0.149), ('capitalist', 0.1322)]
Keywords of article 2 
 [('boyfriend', 0.1686), ('saturday', 0.1386), ('lawyer', 0.1292), ('president', 0.083), ('startup', 0.0811)]
Keywords of article 3 
 [('boyfriend', 0.1917), ('tuesday', 0.1612), ('dropout', 0.1001), ('podcast', 0.0953), ('lawyer', 0.0866)]
Keywords of article 4 
 [('email', 0.1366), ('online', 0.1179), ('monthly', 0.0813), ('login', 0.0735), ('password', 0.0732)]
Keywords of article 5 
 [('2022', 0.2368), ('2021', 0.2246), ('fraudster', 0.1903), ('2020', 0.1695), ('fraudsters', 0.1501)]
Keywords of article 6 
 [('2021', 0.2352), ('investigator', 0.0826), ('latest', 0.0552), ('global', 0.0384), ('banking', 0.0309)]
Keywords of article 7 
 [('australia', 0.2623), ('arrested', 0.2076), ('2021', 0.1914), ('arrest', 0.1814), ('rs26', 0.1774)]
Keywords of article 8 
 [('june', 0.2625), ('scam', 0.1986), ('draftkings', 0.1876), ('whistl

Keywords of article 69 
 [('terrorist', 0.3122), ('terrorism', 0.2334), ('politician', 0.1811), ('laundering', 0.157), ('crime', 0.1188)]
Keywords of article 70 
 [('monaco', 0.2377), ('terrorist', 0.1877), ('2021', 0.1853), ('2020', 0.156), ('july', 0.1551)]
Keywords of article 71 
 [('202103', 0.4084), ('2021', 0.385), ('china', 0.3483), ('beijing', 0.3358), ('july', 0.2917)]
Keywords of article 72 
 [('2021', 0.3891), ('june', 0.3305), ('2020', 0.3212), ('terrorist', 0.2925), ('terrorism', 0.2515)]
Keywords of article 73 
 [('belgrade', 0.2292), ('serbia', 0.2283), ('july', 0.1974), ('terrorist', 0.1781), ('online', 0.1735)]
Keywords of article 74 
 [('terrorist', 0.2064), ('president', 0.1827), ('qaeda', 0.1494), ('manhattan', 0.126), ('terrorism', 0.1243)]
Keywords of article 75 
 [('taliban', 0.3863), ('terrorist', 0.3017), ('afghanistan', 0.2754), ('kidnapping', 0.2365), ('afghan', 0.2159)]
Keywords of article 76 
 [('javascript', 0.4269), ('browser', 0.391), ('cookie', 0.2903),

Keywords of article 140 
 [('monthly', 0.172), ('tomorrow', 0.1713), ('weekly', 0.1645), ('marketwatch', 0.1516), ('2021', 0.1429)]
Keywords of article 141 
 [('tuesday', 0.255), ('monday', 0.1505), ('president', 0.1252), ('cyber', 0.1226), ('cybersecurity', 0.1076)]
Keywords of article 142 
 [('cyber', 0.3381), ('espionage', 0.2543), ('online', 0.2248), ('crime', 0.2028), ('competitive', 0.1963)]
Keywords of article 143 
 [('cybersecurity', 0.2786), ('2025', 0.2467), ('cyber', 0.2125), ('enterprisewide', 0.1904), ('ransomware', 0.1407)]
Keywords of article 144 
 []
Keywords of article 145 
 [('manager', 0.0834), ('competitive', 0.0668), ('elaine', 0.0582), ('cyber', 0.0512), ('hiring', 0.0511)]
Keywords of article 146 
 [('cybercriminals', 0.464), ('cyber', 0.3129), ('phishing', 0.2487), ('bitcoin', 0.2465), ('password', 0.222)]
Keywords of article 147 
 [('cybersecurity', 0.2761), ('nonprofit', 0.2683), ('cyber', 0.2494), ('blog', 0.1812), ('mentorship', 0.1786)]
Keywords of article 

Keywords of article 208 
 [('criminal', 0.1934), ('crime', 0.1707), ('arrested', 0.1486), ('arrest', 0.1448), ('laundering', 0.1204)]
Keywords of article 209 
 [('facebook', 0.0954), ('2017', 0.0532), ('2013', -0.0012), ('december', -0.0103), ('vaccination', -0.014)]
Keywords of article 210 
 [('fraudsters', 0.255), ('aftermarket', 0.2393), ('journalist', 0.1934), ('car', 0.1735), ('advert', 0.139)]
Keywords of article 211 
 []
Keywords of article 212 
 [('crimestoppers', 0.2887), ('instagram', 0.1352), ('newsdesk', 0.1148), ('accountant', 0.1104), ('facebook', 0.0836)]
Keywords of article 213 
 []
Keywords of article 214 
 [('criminal', -0.0074), ('fraudulent', -0.0118), ('nefarious', -0.0224), ('crime', -0.0308), ('mule', -0.0591)]
Keywords of article 215 
 [('terrorist', 0.1584), ('fraudsmart', 0.1507), ('banking', 0.0866), ('2021', 0.0822), ('ireland', 0.0499)]
Keywords of article 216 
 []
Keywords of article 217 
 [('instagram', 0.4242), ('fraudsters', 0.3277), ('netflix', 0.2439)

In [158]:
## RAKE

In [174]:
from rake_nltk import Rake
rake_nltk_var = Rake()
for j in range(len(array_text)):
    rake_nltk_var.extract_keywords_from_text(array_text[j])
    keyword_extracted = rake_nltk_var.get_ranked_phrases()[:5]
    print("Keywords of article", str(j+1), "\n", keyword_extracted)

Keywords of article 1 
 ['many larger health care fraud case', 'political grandee like henry kissinger', 'charismatic elizabeth holmes promised result', 'medium mogul rupert murdoch invested', 'former board member like kissinger']
Keywords of article 2 
 ['falsely claiming theranos could revolutionize medical lab testing', 'alleged `` intimate partner abuse wa relevant', 'theranos inc founder elizabeth holmes ha accused', 'theranos founder elizabeth holmes arrives', 'ramesh `` sunny balwani abused']
Keywords of article 3 
 ['federal fraud trial beginning next week', 'mr balwani wa often physically present', 'abc news true crime podcast', 'separate fraud trial next year', 'theranos received glowing medium coverage']
Keywords of article 4 
 ['numerous website offering unsecured loan', 'sergeant triona brook explained', 'email address password remember', 'official central bank website', 'established financial institution']
Keywords of article 5 
 ['date online spending wa £ 65 billion', '

Keywords of article 81 
 ['profile witness included arkansas supreme court justice rhonda wood', 'bribe former faulkner county circuit judge mike maggio', 'attorney patrick harris declined comment friday', 'central arkansas president tom courtway', 'email seeking comment friday']
Keywords of article 82 
 []
Keywords of article 83 
 []
Keywords of article 84 
 ['big blue taxi service founder datuk shamsubahrin ismail ha claimed trial', 'lawyer muhammad noor azfar muhammad noor azmi', 'prosecuting officer abd muntaqim abdul aziz', 'charge sheet shamsubahrin allegedly accepted', 'case involving azizan mohamed yasin']
Keywords of article 85 
 []
Keywords of article 86 
 ['april 2021 study found six per cent', 'said sanem executive director selim raihan', 'sanem said confidence level regarding recovery', 'remaining 40 per cent ha', 'vaccination took place among dhaka']
Keywords of article 87 
 []
Keywords of article 88 
 ['corruption bureau official raided four residence belonging', 'collec

Keywords of article 163 
 ['pretty big bipartisan majority — 350', 'regularly received confidential information involving actual', 'stock price would jump within le', 'critic questioned whether using confidential information', 'board meeting regarding strategic alternative']
Keywords of article 164 
 ['pretty big bipartisan majority — 350', 'regularly received confidential information involving actual', 'stock price would jump within le', 'critic questioned whether using confidential information', 'board meeting regarding strategic alternative']
Keywords of article 165 
 []
Keywords of article 166 
 ['investment bank advising medivation “ drew close parallel', 'apply insider trading concept — one', 'global healthcare investment banking division', 'corporate insider violates section 10', '“ economically linked ” company']
Keywords of article 167 
 ['market regulator ha also impounded “ ill', 'domestic broking house like ubs', 'sebi ha barred 15 entity', 'nearly r 24 crore', 'included sc

Keywords of article 248 
 ['ezi magbegor scored 14 point apiece', 'katie lou samuelson added 10', 'magbegor wa even hotter', 'mystic came charging back', 'nine straight point early']
Keywords of article 249 
 ['former saturday night live cast member laraine newman testified', 'could frankly color law enforcement —', 'westchester district attorney miriam rocah announced', 'durst legal team member attending', 'longtime friend susan berman —']
Keywords of article 250 
 ['old man wa thrashed', 'kalan station house officer', 'sunil kumar wa attacked', 'wa also thrashed', 'illicit liquor kept']
Keywords of article 251 
 ['popular livestreaming software company owned', 'send redeemable monthly tip notification', 'streamlabs ’ ob software suite', '“ monthly tipping service ”', 'using streamlabs creator subscription seems']
Keywords of article 252 
 ['play eastern conference opponent two time', 'play western conference opponent four time', 'nationally broadcast one pelican matchup –', 'broadcas

In [175]:
# GENSIM

In [178]:
from gensim.summarization import keywords
for j in range(len(array_text)):
    print("Keywords of article", str(j+1), "\n", keywords(array_text[j]))

Keywords of article 1 
 holmes promised
like
company theranos
testimonial likely
venture
testimony
fraud
case
start
billion
dollar
health
promise
manager
managing
testing
test
blood
medium
commission
jailed
jail
investor
rank
successful
time
star
laboratory
patient
street journal
drawing comparison
founder steve
real
murdoch
san
according
Keywords of article 2 
 holmes
court
theranos
lawyer
reuters
filing
balwani abused
testing
university
state
medical test
federal
herrington
trial
expert
connecticut
showed
valley
information
investor
defense
jeff
professor
comment
abusing
abuse
abusive
Keywords of article 3 
 holmes
abuse
abusers
abusive
claim
claimed
claiming
blood startup theranos
fraud trial
lawyer
allegation
alleged
jose
company
investor
control
controlling
controlled
test
testing
revolutionary machine
sharp
decade
state
hard
filing
legal
Keywords of article 4 
 loan
fee
reason
applied
applies
apply
time
financial
website offering
offer
vulnerable
person
address password
Keywords 

Keywords of article 35 
 extraordinary power
legislation
lawful
safeguard
recommendation
recommended
significant
surveillance
crime
australian law
criminal
new
ironside
enables
minister
network activity warrant
particularly
data
technology
pjcis
account
group
amendment
federal government
oversight
authority
judicial
use
organised
organisation
agency
andrew
council
intrusive
approximately
Keywords of article 36 
 
Keywords of article 37 
 crime
police said
parambarai
actor
old
lankan woman
mental
real
mohammed
Keywords of article 38 
 republic
island
kingdom
people
united state
democratic
federative
federated
federal
federation
arab
guinea
territory south
french
new
islamic
china
socialist
principality
principe
samoa
special administrative
commonwealth
luxembourg
grand
british
kong
divoire
fiji
tokelau
swiss confederation
vanuatu venezuela
mayotte
belarus
afghanistan
argentina
faso
namibia
greece
estonia ethiopia
jan
sao tome
great britain
turkmenistan turk
anguilla antarctica
oman
sult

Keywords of article 76 
 support
Keywords of article 77 
 american
intelligence
afghanistan
group
america
killed
killing
force
forced
afghan
province
spying
base
new taliban
drone
spy agency
iskp
kabul
said president
news
counter
biden
strike
suicide
terrorist
close
closely
terror
likely
like
foreign jihadist
year commander
pakistan
central command
pakistani
islamic
target
targeting
missile
underground
increasingly
masoud
lalwani
launched
electronic
bin
potential
according
cia
sending
send
ally
located shamsi
house
white
candidate
iran
longer
multiple
general kenneth mckenzie
suggests raffaello
available
official
turkestan
large
largely
Keywords of article 78 
 india
russian
security
sco
territory
territorial
terror
terrorism
afghanistan
peace
concern
terrorist
focus
virtually
virtual
country
turkistan islamic
trafficking
drug threat
sergey
vladimir
Keywords of article 79 
 measure
recommendation
financial
laundering
deficiency
philippine
amlc
apg
money
risk
country
enhanced
evaluation

Keywords of article 115 
 corruption
corrupt soccer
said
doj
america
world
charged
york
federation
conspiracy charge
fund
state
ill
underprivileged
people
fifa
field office
Keywords of article 116 
 court
land
corruption
corrupt
accused
sale
counsel
observed
state
gain
private
official
revenue
justice
accountability
care
strict
appeal
akbar
submitted
honourable
life
investigation officer
special prosecutor
office
undue
sought
extended lenient
zero
Keywords of article 117 
 
Keywords of article 118 
 secretary
quality
department
engineer
parvatgiri
leader
taking
distributary
official
sudhakar alleged
district
Keywords of article 119 
 
Keywords of article 120 
 uni
corruption
party
democratic alliance
said
Keywords of article 121 
 activity
transaction took
hobby
professional
Keywords of article 122 
 market
nasdaq
surveillance
data
bring
bringing
detect
acquired
dynamic
manage
management
new
crime
support
Keywords of article 123 
 
Keywords of article 124 
 market
global
surveillance
s

Keywords of article 148 
 cyber
secure
securely
said
target
targeting
targeted
security agency
attacked
informed
informing
data
clinical information
regularly
csa
ransomware attack
eye clinic
active
activity
server
organisation
singapore
engineer
engineering
firm
patient
retailer
reported
report
electronic
electronics
group
card
victim
threat
confidentiality
confidential
external
payment
insurer
insurance
anti
locking
lock
locked
bank
banking
extortion
mitigating
potential
branch
malicious
protection
protective
protect
ransom
review
network
asia
Keywords of article 149 
 secure
splunk
information security
fusco
threat
global
operation
effective
said
splk
includes
including
president
sector
continued
monitor
career
trademark
data
bice
turn
organization
today
digex
education group
Keywords of article 150 
 security
secure software
financially
based
financial service
small firm
client
cyber
technology
email
data
significant
cleaner
recent
recently
advice
protection
ftrc
imagine
version
cu

Keywords of article 187 
 credit
identity theft insurance
debt
njmoneyhelp
said
fraudulent
report
reporting
unlock
consulting
trade
financial
police
policy
safety
generally
feather
locking
locks
lock
Keywords of article 188 
 credit
monitor
monitoring
family plan
identity theft protection service
account
include
including
included
protect
fraud
report
reporting
alerting
alert
pro
offer
offering
secure
includes dark
security number
phishing
annual
annually
best
card
new
personal information
insurance
insurer
insure
scanning
scan
address
scam
tax
provide
provides
provided
providing
manager
risk management score
social medium
financial
feature idshield
monthly
website
record
web
individual
password
usually
price
priced
pricing
software
home
company
breach
requesting
request
avoid
avoided
avoiding
bank
email
emailing
legal
bureau
contact
contacting
app
apps
opt
checking
check
formjacking
formjacked
month
money
originally
document
stolen
hacker
coverage
data
wallet
identityforce
ftc receive

Keywords of article 227 
 clearview
data
application
applicant
people
written
algorithm
bipa
waymo
including
white
illinois
mortgage
report
reporter
reported
information
recognition
according
denied
deny
business
autonomous
cent
agency
bear
app
vic
credit
vehicle
group
driver
play
medium
scraped
generate text
generating
yorkshire
northamptonshire
tester program
facial
black
leaked
document
dismiss
dismissed
staff
rebecca
judge pamela
sensor
Keywords of article 228 
 mosbacher
possession
possessing
working
german
crime
warrant
criminal activity
active
offence including money
document
lengthy enquiry
proceeded
proceeding
issue
issued european
address
range
surrey
extradition
prosecution
firearm
crown court
ancillary
licence
Keywords of article 229 
 news
manchester
crime agency
police
august
drive
dealing
cocaine
burglary
subscribe
subscribing
encrypted message
messaging
charity
court
Keywords of article 230 
 kalinin
nca
national crime
new
news
mykhaylov
officer
identity document factor